# 01_DATA_QC_PREP — Quality Check y Splits Estratificados

**Qué hace este notebook (QC básico):**
1. **Carga** cada `driving_log.csv` de los recorridos seleccionados (`RUNS`), admitiendo estructuras con **subcarpetas** (p. ej., `circuito1/vuelta1`, `circuito1/vuelta2`, …).
2. **Normaliza rutas** de imagen (corrige barras invertidas y reduce a rutas relativas tipo `IMG/...`).
3. **Filtra filas inválidas** (imágenes inexistentes en `center/left/right`).
4. Realiza **split estratificado por bins de `steering`** con semilla fija (reproducible).
5. **Escribe**:
   - `data/processed/<run>/canonical.csv`
   - `data/processed/<run>/{train,val,test}.csv`
   - `data/processed/tasks.json` (orden de tareas y rutas a CSV)


> **QC = *Quality Check***: validación y normalización de datos **sin balanceo ni aumentación offline**.  
> Si quieres **balancear** el `train` por bins (con generación de imágenes aumentadas) y **activar la expansión left/right con corrección de `steer_shift`**, usa el cuaderno [`01A_PREP_BALANCED.ipynb`]. **Puedes ejecutar 01A directamente**: incluye su propia fase de *prep* antes de balancear.

---

## 🎯 Objetivos
- Preparar los *splits* `train/val/test` por recorrido
- Generar `tasks.json` para el pipeline *continual*.

## ✅ Prerrequisitos
- Estructura de datos RAW del simulador Udacity, p. ej.:
   - `data/raw/udacity/circuito1/vuelta1/{driving_log.csv, IMG/}`
   - `data/raw/udacity/circuito1/vuelta2/{driving_log.csv, IMG/}`
   - `data/raw/udacity/circuito2/vuelta1/{driving_log.csv, IMG/}`

*(las vueltas pueden variar; el notebook detecta y consolida sus rutas en `canonical.csv`).*

## ⚠️ Salidas
- CSVs canónicos y de split en `data/processed/<run>/`.
- `tasks.json` apuntando a esos CSVs  (orden de tareas para *continual learning*).

> Si más adelante quieres **balanceo offline de `train` por bins** (con aumentación fotométrica) y/o **expansión por cámaras izquierda/derecha (L/R)** con corrección de ángulo, usa `01A_PREP_BALANCED.ipynb` (no requiere ejecutar este notebook previamente).

<a id="toc"></a>

## 🧭 Índice
1. [Entorno y API de preparación](#sec-01)
2. [Parámetros de la preparación](#sec-02)
3. [Ejecutar pipeline y verificar](#sec-03)
4. [Resumen de rutas de splits (vista rápida)](#sec-04)


## <a id="sec-01"></a>

## 1) Entorno y API de preparación

**Objetivo:** preparar el entorno y cargar la API de *prep*.

Esta celda:
- Detecta la raíz del repo (`ROOT`) y la añade a `sys.path`.
- Importa la API de `src.prep.data_prep`:
  - `PrepConfig` — configuración declarativa de la preparación.
  - `run_prep` — ejecuta QC + *splits* estratificados y crea `tasks.json`.
  - `verify_processed_splits` — comprueba que `train/val/test.csv` existen por *run*.
- Define rutas base:
  - `RAW = data/raw/udacity`
  - `PROC = data/processed`

> Aquí **no se ejecuta** todavía la preparación; sólo se deja el entorno listo.

[↑ Volver al índice](#toc)


In [5]:
# %% setup & imports
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys, json
import pandas as pd

ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.prep.data_prep import PrepConfig, run_prep, verify_processed_splits
RAW  = ROOT / "data" / "raw" / "udacity"
PROC = ROOT / "data" / "processed"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <a id="sec-02"></a>

## 2) Parámetros de la preparación

**Objetivo:** declarar los *runs* y la configuración del *split* (QC básico).

- `RUNS`: lista de recorridos a procesar (p. ej., `["circuito1", "circuito2"]`).  
  Cada recorrido puede contener **múltiples subcarpetas de vueltas** (`vuelta1/`, `vuelta2/`, …) que el *prep* consolidará en `canonical.csv`.
- `use_left_right=False`: en este cuaderno mantenemos un **QC sin expansión L/R** (no triplica filas ni aplica `steer_shift`).  
  > Si deseas activar **expansión L/R + corrección de ángulo** y **balanceo offline por bins**, emplea `01A_PREP_BALANCED.ipynb` (recomendado para comparativas).
- `bins`: número de contenedores para **estratificar `steering`** en los *splits* (mitiga sesgo a “recta”).
- `train/val`: proporciones del *split* (test se infiere).
- `seed`: fija reproducibilidad del *split*.

> `target_per_bin` y `cap_per_bin` **no se usan aquí**. Son parámetros del **balanceo offline por imágenes** en `01A_PREP_BALANCED.ipynb`.

[↑ Volver al índice](#toc)


In [ ]:
# Parámetros de la preparación
RUNS = ["circuito1", "circuito2"]

CFG = PrepConfig(
    root=ROOT,
    runs=RUNS,
    use_left_right=False,  # QC + split básico (sin expansión)
    steer_shift=0.2,       # no aplica si use_left_right=False
    bins=50,
    train=0.70,
    val=0.15,
    seed=42,
    target_per_bin=None,   # <- no balancea aquí
    cap_per_bin=None,
)
CFG

PrepConfig(root=PosixPath('/home/cesar/proyectos/TFM_SNN'), runs=['circuito1', 'circuito2'], use_left_right=False, steer_shift=0.2, bins=21, train=0.7, val=0.15, seed=42, target_per_bin=None, cap_per_bin=None)

## <a id="sec-03"></a>
## 3) Ejecutar *prep* y verificar

**Objetivo:** correr el pipeline de QC + *splits* y validar la existencia de los CSV.

Qué hace `run_prep(CFG)`:
- Lee y **normaliza** cada `driving_log.csv` (admite subcarpetas de vueltas) → genera `canonical.csv`.
- Crea *splits* **estratificados** (`train.csv`, `val.csv`, `test.csv`) con semilla fija.
- Construye/actualiza `data/processed/tasks.json` con el **orden de tareas** y rutas a CSV por *run*.

Qué hace `verify_processed_splits(PROC, RUNS)`:
- Comprueba que `train/val/test.csv` **existen** para cada *run*.
- Si falta algo, **lanza una excepción** con detalle (ayuda a detectar problemas de rutas o imágenes faltantes).

> **Idempotencia:** puedes **re-ejecutar** esta celda tantas veces como quieras. Se regenerarán los CSVs y se actualizará `tasks.json` de forma consistente con lo indicado en `RUNS`.

[↑ Volver al índice](#toc)


In [7]:
# Ejecutar prep + verificar
manifest = run_prep(CFG)
print("OK:", PROC/"prep_manifest.json")

# Verificación: existen train/val/test por run
verify_processed_splits(PROC, RUNS)
print("OK: splits 'train/val/test' encontrados.")


OK: /home/cesar/proyectos/TFM_SNN/data/processed/prep_manifest.json
OK: splits 'train/val/test' encontrados.


## <a id="sec-04"></a>
## 4) Resumen de rutas de *splits* (vista rápida)

**Objetivo:** visualizar de un vistazo qué ficheros usará el pipeline *continual*.

Esta celda:
- Lee `data/processed/tasks.json`.
- Muestra una tabla con columnas `run`, `train_csv`, `val_csv` y `test_csv`.

> Si las rutas no son las esperadas, revisa:
> - La **lista `RUNS`** de la celda 2,
> - La **estructura** de `data/raw/udacity/<run>/vuelta*/`,
> - Y que `IMG/` contenga las imágenes referenciadas.

> **Nota sobre balanceo:** este notebook **no** crea `tasks_balanced.json`.  
> Si quieres usar *splits* balanceados y/o expansión L/R, ejecuta `01A_PREP_BALANCED.ipynb` (puedes lanzarlo directamente; incluye su propia fase de *prep* antes del balanceo).

[↑ Volver al índice](#toc)


In [8]:
# Resumen de rutas de splits (visualización)
tasks_json = json.loads((PROC/"tasks.json").read_text(encoding="utf-8"))
pd.DataFrame({
    "run": tasks_json["tasks_order"],
    "train_csv": [tasks_json["splits"][r]["train"] for r in tasks_json["tasks_order"]],
    "val_csv":   [tasks_json["splits"][r]["val"]   for r in tasks_json["tasks_order"]],
    "test_csv":  [tasks_json["splits"][r]["test"]  for r in tasks_json["tasks_order"]],
})


,run,train_csv,val_csv,test_csv
0,circuito1,/home/cesar/proyectos/TFM_SNN/data/processed/c...,/home/cesar/proyectos/TFM_SNN/data/processed/c...,/home/cesar/proyectos/TFM_SNN/data/processed/c...
1,circuito2,/home/cesar/proyectos/TFM_SNN/data/processed/c...,/home/cesar/proyectos/TFM_SNN/data/processed/c...,/home/cesar/proyectos/TFM_SNN/data/processed/c...
